In [ ]:
from neuron import h, crxd as rxd, gui
from matplotlib import pyplot, animation
from IPython.display import HTML

rxd.options.enable.extracellular = True
rxd.nthread(4)

In [ ]:
class Cell:
    def __init__(self,filename):
        h.load_file('import3d.hoc')
        cell = h.Import3d_SWC_read()
        cell.input(filename)
        h.Import3d_GUI(cell, 0)
        i3d = h.Import3d_GUI(cell, 0)
        i3d.instantiate(self)
        for sec in self.all:
            sec.nseg = 1 + 10 * int(sec.L / 5)
            sec.insert('steady_k')
    def extrema(self):
        h.define_shape()
        xlo = ylo = zlo = xhi = yhi = zhi = None
        for sec in self.all:
            n3d = int(h.n3d(sec=sec))
            xs = [h.x3d(i, sec=sec) for i in xrange(n3d)]
            ys = [h.y3d(i, sec=sec) for i in xrange(n3d)]
            zs = [h.z3d(i, sec=sec) for i in xrange(n3d)]
            my_xlo, my_ylo, my_zlo = min(xs), min(ys), min(zs)
            my_xhi, my_yhi, my_zhi = max(xs), max(ys), max(zs)
            if xlo is None:
                xlo, ylo, zlo = my_xlo, my_ylo, my_zlo
                xhi, yhi, zhi = my_xhi, my_yhi, my_zhi
            else:
                xlo, ylo, zlo = min(xlo, my_xlo), min(ylo, my_ylo), min(zlo, my_zlo)
                xhi, yhi, zhi = max(xhi, my_xhi), max(yhi, my_yhi), max(zhi, my_zhi)
        return xlo, ylo, zlo, xhi, yhi, zhi



In [ ]:
mycell = Cell('c91662.swc')

xlo, ylo, zlo, xhi, yhi, zhi = mycell.extrema()
padding = 50
ecs = rxd.Extracellular(xlo - padding, 
                        ylo - padding, 
                        zlo - padding,
                        xhi + padding,
                        yhi + padding,
                        zhi + padding,
                        dx=10)

k = rxd.Species(ecs, d=2.62, name='k', charge=1, initial=0)
h.dt = 10

In [ ]:
h.finitialize()
fig = pyplot.figure()
im = pyplot.imshow(k[ecs].states3d.mean(2),vmin=0,vmax=0.05)
pyplot.axis('off')

def init():
    im.set_data(k[ecs].states3d.mean(2))
    return [im]

def animate(i):
    h.fadvance()
    im.set_data(k[ecs].states3d.mean(2))
    return [im]

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=500, interval=10)
HTML(anim.to_html5_video())